In [1]:
import pandas as pd
import numpy as np
import re

## ler arquivo json em pandas
df = pd.read_json('C:\\Users\\mayco\\Documents\\projetos\\data-pantanaldev\\label-studio\\data\\export\\project-1-at-2023-04-27-03-26-58dd1708.json')
## expandir coluna annotations
df = pd.concat([df.drop(['annotations'], axis=1), df['annotations'].apply(pd.Series)], axis=1)
## expandir coluna 0 e renomear para annotations
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna result e renomear para result
df = pd.concat([df.drop(['result'], axis=1), df['result'].apply(pd.Series)], axis=1)
## expandir coluna 0 e renomear para result
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna value e renomear para value
df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
## dropar choices nulos
df = df.dropna(subset=['choices'])
## obter choices 
df['choices'] = df['choices'].apply(lambda x: x[0])


## expandir coluna data
df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)
padrao_data_cepea = r"Cepea, \d{2}/\d{2}/\d{4} - "
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_data_cepea, '', x))

## conter apenas soja no titulo
#df = df[df['titulo'].str.contains('SOJA')]

def preprocess_text(text):
    # Remover pontuações
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenização
    words = word_tokenize(text.lower())
    # Remover stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['noticia'] = df['noticia'].apply(preprocess_text)


columns_to_select = ['id', 'data', 'noticia', 'titulo', 'choices']
df = df[columns_to_select]
df_modelo = df[:122]
df_validacao = df[122:]



NameError: name 'word_tokenize' is not defined

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

X = df_modelo['noticia']
y = df_modelo['choices']

# Define os parâmetros que serão testados no GridSearch
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000),
    'vect__ngram_range': ((1, 1), (1, 2)),  
    'clf__max_depth': [10, 50, 100, None],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4]
}

# Cria um pipeline com o TfidfVectorizer e a DecisionTreeClassifier
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', DecisionTreeClassifier())
])

# Executa o GridSearch usando a cross-validation com 5 folds
grid_search = GridSearchCV(pipeline, parameters, cv=2, n_jobs=-1, verbose=1)

# Divide em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Executa o GridSearch no conjunto de treino
grid_search.fit(X_train, y_train)

# Imprime o relatório de classificação com os melhores parâmetros encontrados
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=0))

# Imprime o melhor conjunto de parâmetros encontrados
print(grid_search.best_params_)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder

# Transforma o texto em vetores numéricos com o TfidfVectorizer
vect = TfidfVectorizer()
X_train_vect = vect.fit_transform(df_modelo['noticia'])

# Converte as labels para valores numéricos
le = LabelEncoder()
y_train = le.fit_transform(df_modelo['choices'])

# Converte os dados para tensores do PyTorch
tensor_X_train = torch.tensor(X_train_vect.toarray(), dtype=torch.float32)
tensor_y_train = torch.tensor(y_train, dtype=torch.long)

# Cria o DataLoader para carregar os dados na rede neural
batch_size = 16
train_dataset = TensorDataset(tensor_X_train, tensor_y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define a arquitetura da rede neural
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

# Define os parâmetros da rede neural
input_size = X_train_vect.shape[1]
hidden_size = 10
output_size = len(set(df_modelo['choices']))

# Cria a rede neural
net = Net(input_size, hidden_size, output_size)

# Treina a rede neural
num_epochs = 100
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        inputs, labels = data
        
        # Faz uma forward pass pela rede neural
        outputs = net(inputs)
        
        # Calcula a função de perda
        loss = criterion(outputs, labels)
        
        # Faz uma backward pass para calcular os gradientes
        optimizer.zero_grad()
        loss.backward()
        
        # Atualiza os parâmetros da rede neural
        optimizer.step()

## validação
# Transforma o texto em vetores numéricos com o TfidfVectorizer
X_valid_vect = vect.transform(df_validacao['noticia'])

# Converte as labels para valores numéricos
y_valid = le.transform(df_validacao['choices'])

# Converte os dados para tensores do PyTorch
tensor_X_valid = torch.tensor(X_valid_vect.toarray(), dtype=torch.float32)
tensor_y_valid = torch.tensor(y_valid, dtype=torch.long)

# Cria o DataLoader para carregar os dados na rede neural
valid_dataset = TensorDataset(tensor_X_valid, tensor_y_valid)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

# Calcula a acurácia da rede neural
correct = 0
total = 0
with torch.
print('Acurácia da rede neural: %d %%' % (100 * correct / total))
